In [ ]:
pip install pycountry

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 29.5 MB/s eta 0:00:00


In [ ]:
#All imports go into here ONLY.
from   sklearn.feature_extraction.text import CountVectorizer

from   sklearn.preprocessing   import StandardScaler
from   sklearn.pipeline        import Pipeline
from   sklearn.preprocessing   import OneHotEncoder
from   sklearn.preprocessing   import LabelEncoder
from   sklearn.utils           import sparsefuncs
from   sklearn.metrics         import f1_score
from   sklearn.decomposition   import PCA
from   sklearn.decomposition   import TruncatedSVD
from   sklearn.datasets        import load_iris
from   sklearn.linear_model    import Perceptron
from   sklearn.model_selection import train_test_split
from   sklearn.naive_bayes     import BernoulliNB
from   sklearn.naive_bayes     import MultinomialNB
from   sklearn.metrics         import accuracy_score, confusion_matrix
from   sklearn.model_selection import GridSearchCV
from   sklearn.pipeline        import make_pipeline
from   sklearn.metrics         import mean_squared_error
from   sklearn.metrics         import make_scorer

from   scipy                   import sparse
from   subprocess              import check_output

import numpy             as np
import pandas            as pd
import matplotlib.pyplot as plt
import seaborn           as sns
import pycountry         as pycry

In [48]:
country_names = np.array( [ country.name for country in pycry.countries ] )

raw_text_input_df = pd.read_excel(
                                    "/content/drive/MyDrive/TextFingerprinter/raw_text_input.xlsx",
                                    #true_values="Y",
                                    #false_values="N",
                                    #keep_default_na = False,
                                    #na_values = "nth",
                                    #index_col = "FirstName"
                    )

#x_train, x_test, y_train, y_test = train_test_split(raw_text_input_df["RAW_TEXT"], raw_text_input_df["RESULT"], test_size=0.2, random_state=10)


country_vectorizer = CountVectorizer(

                                       #input      = x_train,
                                       lowercase  = False,
                                       vocabulary = country_names
)


pipeline_centered_SVD  = Pipeline( [

                                     ( "counter_vectorizer", CountVectorizer() ),
                                     (                "pca", PCA()             ),
                                     #(          "Bernoulli", BernoulliNB()     ),
                                 ]

                        )

pipeline_truncated_SVD  = Pipeline( [
                                      ( "counter_vectorizer", CountVectorizer() ),
                                      (      "truncated_SVD", TruncatedSVD()    ),

                                  ]

                         )

label_encoder          = LabelEncoder()

param_dict             =  {

                                 "counter_vectorizer__lowercase"  : False,
                                 "counter_vectorizer__vocabulary" : country_names,
                                "counter_vectorizer__ngram_range" : (1,4)

                        }

param_dict_trunc_SVD =  {

                                "counter_vectorizer__lowercase"  : False,
                                "counter_vectorizer__vocabulary" : country_names,
                               "counter_vectorizer__ngram_range" : (1,4),
                                   "truncated_SVD__n_components" : 3,

            }

result_array          = np.array( [ "Y" , "N" ] )
#rmse = make_scorer(mean_squared_error, squared=False)


In [49]:
pipeline_centered_SVD.set_params(**param_dict)
countries_vectorized_pipeline = pipeline_centered_SVD.fit_transform( raw_text_input_df["RAW_TEXT"] )

print( countries_vectorized_pipeline[4] )

[-9.58334407e-02 -1.16642377e-16 -2.77578023e-16 -1.79271486e-01
 -2.42928300e-17  3.27862815e-18 -4.30259000e-32 -2.53171411e-33
 -1.04501371e-32  7.65439251e-49]


In [50]:
pipeline_truncated_SVD.set_params(**param_dict_trunc_SVD)
countries_vectorized_pipeline = pipeline_truncated_SVD.fit_transform( raw_text_input_df["RAW_TEXT"] )

print( countries_vectorized_pipeline[2] )
print( countries_vectorized_pipeline[4] )


[-2.16832338e-17  4.47068630e-01  1.13859744e-02]
[0. 0. 0.]


#Ensures the correctness of results_vectorized

In [53]:
for result in raw_text_input_df["RESULT"]:
  print( result )

label_encoder.fit( result_array )
results_vectorized      = label_encoder.transform( raw_text_input_df["RESULT"] )

#print( result )

Y
N
Y
N
N
N
N
N
N
Y
Y


In [64]:
#grid_search_score =

model = GridSearchCV(
                      estimator  = BernoulliNB(),
                      param_grid = { "alpha": [0.1, 0.5, 1.0, 2.0], "binarize": [0.0, 0.5, 1.0]  },
                      n_jobs     = -1,
                      cv         = 5,
                      scoring    = "f1_macro"

        )

model.fit(
                      countries_vectorized_pipeline,
                      results_vectorized
)



/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(


GridSearchCV(cv=5, estimator=BernoulliNB(), n_jobs=-1,
             param_grid={'alpha': [0.1, 0.5, 1.0, 2.0],
                         'binarize': [0.0, 0.5, 1.0]},
             scoring='f1_macro')

In [65]:
print("--------GridSearchCV_F1_Score-----------")
print(model.best_score_)

--------GridSearchCV_F1_Score-----------
1.0


In [66]:
print("--------GridSearchCV_best_params-----------")
print(model.best_params_)

--------GridSearchCV_best_params-----------
{'alpha': 0.1, 'binarize': 0.0}


In [84]:
testing_countries = pipeline_truncated_SVD.transform( np.array( [  "Poland" , "China" , "Slovakia" , "Spain"  ] )  )

print( model.predict( testing_countries  ) )

[0 1 1 0]


#PCA Analysis
When SVD == True